In [ ]:
# source: https://github.com/ollama/ollama/blob/main/docs/tutorials/langchainpy.md

from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA

ollama = Ollama(base_url='http://localhost:11434', model="llama2")

loader = WebBaseLoader("https://www.gutenberg.org/files/1727/1727-h/1727-h.htm")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(data)

oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
# store the emeddings locally - maybe qdrant?
vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)

question = "Who is Neleus and who is in Neleus' family?"
print(question)
docs = vectorstore.similarity_search(question)

qachain = RetrievalQA.from_chain_type(ollama, retriever=vectorstore.as_retriever())
print(qachain.invoke({"query": question}))